Ignis: Latent Dirichlet Allocation
============

In [8]:
import ignis

In [9]:
# Jupyter notebook setup
import ipywidgets as widgets
from IPython.core.display import display, HTML

# Custom styling:
# - Prevent vertical scrollbars in output subareas
# - Resize to fit pyLDAvis visualisations without causing other cells to overflow
style = """
<style>
   div.cell > div.output_wrapper > div.output.output_scroll {
     height: auto;
   }
   .jupyter-widgets-output-area .output_scroll {
        height: unset;
        border-radius: unset;
        -webkit-box-shadow: unset;
        box-shadow: unset;
    }
    .jupyter-widgets-output-area, div.output_stdout, div.output_result  {
        height: auto;
        max-height: 60em;
        overflow-y: auto;
    }
</style>
<style>
    #notebook-container { width: 1370px !important; }
    div.output_area { width: unset !important; }
</style>
"""
display(HTML(style))

Model training (LDA)
----

Load from an `ignis.Corpus`, add the processed docs to an LDA model, and train it.

The random seed and parallelisation can both affect results, so setting the seed and number of workers is necessary for reproducibility.

In [10]:
corpus = ignis.load_corpus("bbc.corpus")

With the current public version of `pyLDAvis`, (2.1.2), preparing the visualisation data takes very long with recent versions of `pandas` (>0.23.4).  We have an option here to use an optimised version of the preparation function built into Ignis.

In [11]:
use_optimised = True

In [12]:
model_options = {"term_weighting": "pmi", "verbose": True, "workers": 10}
vis_options = {"verbose": True, "use_optimised": use_optimised}

Ignis can suggest a recommended number of topics to use for LDA, based on the coherence scores of a range of lightly-trained models.

In [13]:
best_k = ignis.suggest_num_topics(
    corpus, model_options=model_options, verbose=True
)

  0%|          | 0/900 [00:00<?, ?it/s]

Training 9 mini-models to suggest a suitable number of topics between 2 and 10...(2118 documents, 100 iterations each)


100%|██████████| 900/900 [00:12<00:00, 69.44it/s]

Suggested topic count: 4	Coherence: -1.5316458452546042 (always negative, closer to 0 is better)


In [14]:
results = ignis.train_model(
    corpus,
    model_options=dict(model_options, k=best_k, until_max_coherence=True),
)

Training model on 2118 documents:
{'term_weighting': 'pmi', 'k': 4, 'seed': 11399, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': False, 'until_max_coherence': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:05<00:00, 185.81it/s, Log-likelihood=-32.40526]



Continuing to train until maximum coherence.



700it [00:09, 71.18it/s, Coherence=-1.76571]


Best coherence: -1.71546 (Starting: -1.84071
Docs: 2118, Vocab size: 19290, Total Words: 412636
Model training complete. (16.057s)


In [15]:
results.init_vis("pyldavis")
results.show_visualisation()

In [ ]:
# results.export_visualisation("bbc_results")

## Saving results, using the automated labeller

In [16]:
results.save("bbc.aurum")
results = ignis.load_results("bbc.aurum")

In [17]:
results.init_labeller("tomotopy", verbose=True)

Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


## Exploring documents that "belong" to a given topic

Because topics are distributions over words and documents are *distributions* over topics, documents don't belong to individual topics per se; every topic is represented in every document with some probability.

We therefore have to specify how many of the document's top `n` topics we want to check for the actual topic we're interested in.

This is especially significant for topic models that use a term weighting scheme, because all the common words (i.e., what we might consider stopwords) tend to get grouped into a single large topic; if we only consider each document's single most probable topic, we will unintentionally exclude documents which have this "stopwords" topic as their top topic.  (This is, however, not always a given; one would need to check the model's output and iterate from there).

Alternatively, we can specify `ignore_topics` when slicing to completely ignore a given topic when classifying documents.

In [18]:
results.nb_explore_topics()

interactive(children=(IntSlider(value=1, description='Topic', max=4, min=1), Output()), _dom_classes=('widget-…

<function ignis.aurum.Aurum.nb_explore_topics.<locals>.show_topic(topic_id=1)>

Slicing and iteration
--------
After seeing what the main topics might be, we can slice the initial corpus further and re-run topic modelling to get better resolution.

In [21]:
# Try zooming in on the entertainment topic
entertainment_slice = results.slice_by_topic(topic_id=2)
best_k_2 = results.resuggest_num_topics(corpus_slice=entertainment_slice)

Training LDA model on 332 documents:
{'term_weighting': 'pmi', 'k': 5, 'seed': 11399, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 100, 'update_every': 100, 'until_max_ll': False, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 100/100 [00:00<00:00, 413.22it/s, Log-likelihood=-28.55111]

Docs: 332, Vocab size: 8622, Total Words: 53438
Model training complete. (0.245s)
Training LDA model on 332 documents:
{'term_weighting': 'pmi', 'k': 6, 'seed': 11399, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 100, 'update_every': 100, 'until_max_ll': False, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}




100%|██████████| 100/100 [00:00<00:00, 427.39it/s, Log-likelihood=-28.48563]

Docs: 332, Vocab size: 8622, Total Words: 53438
Model training complete. (0.236s)
Training LDA model on 332 documents:
{'term_weighting': 'pmi', 'k': 7, 'seed': 11399, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 100, 'update_every': 100, 'until_max_ll': False, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}



...

Docs: 332, Vocab size: 8622, Total Words: 53438
Model training complete. (0.333s)
Training LDA model on 332 documents:
{'term_weighting': 'pmi', 'k': 14, 'seed': 11399, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 100, 'update_every': 100, 'until_max_ll': False, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}




100%|██████████| 100/100 [00:00<00:00, 303.04it/s, Log-likelihood=-27.77230]

Docs: 332, Vocab size: 8622, Total Words: 53438
Model training complete. (0.334s)
Training LDA model on 332 documents:
{'term_weighting': 'pmi', 'k': 15, 'seed': 11399, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 100, 'update_every': 100, 'until_max_ll': False, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}




100%|██████████| 100/100 [00:00<00:00, 299.40it/s, Log-likelihood=-27.69840]

Docs: 332, Vocab size: 8622, Total Words: 53438
Model training complete. (0.336s)


In [22]:
entertainment_model = results.retrain_model(
    corpus_slice=entertainment_slice, model_options={"k": best_k_2[0]}
)
entertainment_model.show_visualisation()

Training LDA model on 332 documents:
{'term_weighting': 'pmi', 'k': 5, 'seed': 11399, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:02<00:00, 482.17it/s, Log-likelihood=-27.91753]


Continuing to train until maximum log-likelihood.
(N.B.: This may not correlate with increased interpretability)




2000it [00:05, 396.98it/s, Log-likelihood=-27.84497]

Docs: 332, Vocab size: 8622, Total Words: 53438
Model training complete. (7.268s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.
Preparing LDA visualisation...

 Done. (0.310s)


In [ ]:
# entertainment_model.export_visualisation("bbc_results_2")

The position of each topic cluster on the graph is not intrinsically informative per se (being simply the result of some specified dimensionality-reducing technique), but if we want we can run the modelling algorithm with a different random seed and see if we get a more meaningful set of topics.

(We can also override any of the previously set options.)

In [24]:
new_model_options = {"seed": 3256}
entertainment_model_2 = entertainment_model.retrain_model(
    model_options=new_model_options
)
entertainment_model_2.show_visualisation()

Training LDA model on 332 documents:
{'term_weighting': 'pmi', 'k': 5, 'seed': 3256, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:02<00:00, 486.98it/s, Log-likelihood=-27.91635]


Continuing to train until maximum log-likelihood.
(N.B.: This may not correlate with increased interpretability)




1700it [00:03, 441.79it/s, Log-likelihood=-27.84439]

Docs: 332, Vocab size: 8622, Total Words: 53438
Model training complete. (6.050s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.
Preparing LDA visualisation...

 Done. (0.308s)


Notice that the topic words remain more or less consistent across different training runs, even though their positions in the visualisation change with each iteration.

In [25]:
# entertainment_model_2.export_visualisation("bbc_results_2a")

In [26]:
entertainment_model_2.nb_explore_topics()

interactive(children=(IntSlider(value=1, description='Topic', max=5, min=1), Output()), _dom_classes=('widget-…

<function ignis.aurum.Aurum.nb_explore_topics.<locals>.show_topic(topic_id=1)>

In addition to simply slicing by topic, we can also explicitly search the whole corpus for documents that contain certain tokens, in case we want to be absolutely sure we got all the documents that mention certain words or phrases.

In [27]:
# Topics that seem to deal with films
film_topic = entertainment_model_2.slice_by_topic(topic_id=3)

# Tokens that are related to films, doing a full-text search through the entire corpus (not just within the current results)
by_token_slice = entertainment_model_2.slice_by_tokens(
    ["film", "films", "movie", "cinema"], include_root=True
)

concat_slice = film_topic.concat(by_token_slice)

concat_model = entertainment_model_2.retrain_model(
    concat_slice, model_options=model_options
)
concat_model.show_visualisation()

Training LDA model on 288 documents:
{'term_weighting': 'pmi', 'k': 5, 'seed': 3256, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:02<00:00, 475.96it/s, Log-likelihood=-27.52412]


Continuing to train until maximum log-likelihood.
(N.B.: This may not correlate with increased interpretability)




1200it [00:02, 447.43it/s, Log-likelihood=-27.48300]

Docs: 288, Vocab size: 9313, Total Words: 56347
Model training complete. (4.927s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.
Preparing LDA visualisation...

 Done. (0.307s)


In [ ]:
# concat_model.export_visualisation("bbc_results_3")

In [28]:
# Topic related to film awards
concat_model.nb_explore_topics()

interactive(children=(IntSlider(value=1, description='Topic', max=5, min=1), Output()), _dom_classes=('widget-…

<function ignis.aurum.Aurum.nb_explore_topics.<locals>.show_topic(topic_id=1)>